In [0]:
#credentials
consumer_key = "" 
consumer_secret = ""
access_token = ""
access_token_secret = ""

import tweepy
from textblob import TextBlob
import re
import pandas as pd

In [0]:
#userID whose tweets to be extracted
userID = "realDonaldTrump"

In [106]:
#extrating tweets
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

tweets = api.user_timeline(screen_name=userID, 
                           # 200 is the maximum allowed count
                           count=200,
                           include_rts = False,
                           # Necessary to keep full_text 
                           # otherwise only the first 140 words are extracted
                           tweet_mode = 'extended'
                           )
for info in tweets[:3]:
     print("ID: {}".format(info.id))
     print(info.created_at)
     print(info.full_text)
     print("\n")

ID: 1269452057737641984
2020-06-07 02:12:10
.@CNN &amp; MSDNC are doing everything possible to “inflame” the crowd. Fortunately they have a very small audience!


ID: 1269451304503574528
2020-06-07 02:09:10
Much smaller crowd in D.C. than anticipated. National Guard, Secret Service, and D.C. Police have been doing a fantastic job. Thank you!


ID: 1269400770472001539
2020-06-06 22:48:22
LAW &amp; ORDER!




In [107]:
#extracting tweets >200
all_tweets = []
all_tweets.extend(tweets)
oldest_id = tweets[-1].id
while True:
    tweets = api.user_timeline(screen_name=userID, 
                           # 200 is the maximum allowed count
                           count=200,
                           include_rts = False,
                           max_id = oldest_id - 1,
                           # Necessary to keep full_text 
                           # otherwise only the first 140 words are extracted
                           tweet_mode = 'extended'
                           )
    if len(tweets) == 0:
        break
    oldest_id = tweets[-1].id
    all_tweets.extend(tweets)
    print('N of tweets downloaded till now {}'.format(len(all_tweets)))

N of tweets downloaded till now 138
N of tweets downloaded till now 224
N of tweets downloaded till now 341
N of tweets downloaded till now 462
N of tweets downloaded till now 550
N of tweets downloaded till now 616
N of tweets downloaded till now 714
N of tweets downloaded till now 805
N of tweets downloaded till now 886
N of tweets downloaded till now 979
N of tweets downloaded till now 1065
N of tweets downloaded till now 1173
N of tweets downloaded till now 1215
N of tweets downloaded till now 1288
N of tweets downloaded till now 1388
N of tweets downloaded till now 1451


In [108]:
#saving in csv format
tweet.encode("utf-8")

#run the subsequent blocks

AttributeError: ignored

In [109]:
#transform the tweepy tweets into a array 	
from pandas import DataFrame
outtweets = [[tweet.id_str, 
              tweet.created_at, 
              tweet.favorite_count, 
              tweet.retweet_count, 
              tweet.full_text.encode("utf-8").decode("utf-8")] 
             for idx,tweet in enumerate(all_tweets)]
df = DataFrame(outtweets,columns=["id","created_at","favorite_count","retweet_count", "text"])
df.to_csv('%s_tweets.csv' % userID,index=False)
df.head()

,id,created_at,favorite_count,retweet_count,text
0,1269452057737641984,2020-06-07 02:12:10,103445,24113,.@CNN &amp; MSDNC are doing everything possibl...
1,1269451304503574528,2020-06-07 02:09:10,144531,27609,Much smaller crowd in D.C. than anticipated. N...
2,1269400770472001539,2020-06-06 22:48:22,249734,47265,LAW &amp; ORDER!
3,1269396484048465921,2020-06-06 22:31:20,183316,33820,Wow! 96% Approval Rating in the Republican Par...
4,1269357552854327296,2020-06-06 19:56:38,55492,19613,https://t.co/hJSsx86Azp


In [0]:
#cleaning the tweets
features= df.iloc[:,4].values
processed_features = []
for sentence in range(0, len(features)):
    # Remove all the special characters
    processed_feature = re.sub(r'\W', ' ', str(features[sentence]))

    # remove all single characters
    processed_feature= re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_feature)

    # Remove single characters from the start
    processed_feature = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_feature) 

    # Substituting multiple spaces with single space
    processed_feature = re.sub(r'\s+', ' ', processed_feature, flags=re.I)

    # Removing prefixed 'b'
    processed_feature = re.sub(r'^b\s+', '', processed_feature)

    # Converting to Lowercase
    processed_feature = processed_feature.lower()

    processed_features.append(processed_feature)

In [115]:
processed_features

[' cnn amp msdnc are doing everything possible to inflame the crowd fortunately they have very small audience ',
 'much smaller crowd in c than anticipated national guard secret service and c police have been doing fantastic job thank you ',
 'law amp order ',
 'wow 96 approval rating in the republican party and believe this was before the great jobs numbers yesterday thank you ',
 'https co hjssx86azp',
 'https co faypsssqcs',
 'riot gear or military control is not necessary because antifa amp other wacko groups of anarchists aren present to cause trouble incredible people thank you maine https co kgocoqormg',
 'great new book by pastordscott called nothing to lose the pastor is winner and great guy much can be learned from him buy the book make him big deal author ',
 ' this is flat out lie they the media are making things up charleshurt ingrahamangle the lamestream media is out of control it would be impossible to fully explain how dishonest they are ',
 'crazy https co 8usgj6yyx4',

In [111]:
#inserting processed_features as new column
df_1=pd.DataFrame(processed_features)
df_a=pd.DataFrame(df)
dataset=pd.concat([df_a,df_1],axis=1)
dataset.rename(columns={0:'processed_text'},inplace=True)
dataset.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1451 entries, 0 to 1450
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              1451 non-null   object        
 1   created_at      1451 non-null   datetime64[ns]
 2   favorite_count  1451 non-null   int64         
 3   retweet_count   1451 non-null   int64         
 4   text            1451 non-null   object        
 5   processed_text  1451 non-null   object        
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 68.1+ KB


In [112]:
#adding new column for sentiments
dataset["sentiments"]=""
dataset

,id,created_at,favorite_count,retweet_count,text,processed_text,sentiments
0,1269452057737641984,2020-06-07 02:12:10,103445,24113,.@CNN &amp; MSDNC are doing everything possibl...,cnn amp msdnc are doing everything possible t...,
1,1269451304503574528,2020-06-07 02:09:10,144531,27609,Much smaller crowd in D.C. than anticipated. N...,much smaller crowd in c than anticipated natio...,
2,1269400770472001539,2020-06-06 22:48:22,249734,47265,LAW &amp; ORDER!,law amp order,
3,1269396484048465921,2020-06-06 22:31:20,183316,33820,Wow! 96% Approval Rating in the Republican Par...,wow 96 approval rating in the republican party...,
4,1269357552854327296,2020-06-06 19:56:38,55492,19613,https://t.co/hJSsx86Azp,https co hjssx86azp,
...,...,...,...,...,...,...,...
1446,1234909058529943552,2020-03-03 18:30:37,47534,10750,I commend the hard work of Senator @SteveDaine...,i commend the hard work of senator stevedaines...,
1447,1234894925004779520,2020-03-03 17:34:28,50831,12310,#PrayForNashville https://t.co/cja0lMG6VC,prayfornashville https co cja0lmg6vc,
1448,1234893805389844481,2020-03-03 17:30:01,40324,9603,It was my great honor to address the National ...,it was my great honor to address the national ...,
1449,1234869067892305923,2020-03-03 15:51:43,77590,18805,The Federal Reserve is cutting but must furthe...,the federal reserve is cutting but must furthe...,


In [113]:
for i in range(0,1450):
  blob=TextBlob(dataset['processed_text'][i])
  if blob.sentiment.polarity>0:
    dataset['sentiments'][i]="+1"
  else:
    dataset['sentiments'][i]="-1"
     
            

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [114]:
dataset

,id,created_at,favorite_count,retweet_count,text,processed_text,sentiments
0,1269452057737641984,2020-06-07 02:12:10,103445,24113,.@CNN &amp; MSDNC are doing everything possibl...,cnn amp msdnc are doing everything possible t...,+1
1,1269451304503574528,2020-06-07 02:09:10,144531,27609,Much smaller crowd in D.C. than anticipated. N...,much smaller crowd in c than anticipated natio...,-1
2,1269400770472001539,2020-06-06 22:48:22,249734,47265,LAW &amp; ORDER!,law amp order,-1
3,1269396484048465921,2020-06-06 22:31:20,183316,33820,Wow! 96% Approval Rating in the Republican Par...,wow 96 approval rating in the republican party...,+1
4,1269357552854327296,2020-06-06 19:56:38,55492,19613,https://t.co/hJSsx86Azp,https co hjssx86azp,-1
...,...,...,...,...,...,...,...
1446,1234909058529943552,2020-03-03 18:30:37,47534,10750,I commend the hard work of Senator @SteveDaine...,i commend the hard work of senator stevedaines...,-1
1447,1234894925004779520,2020-03-03 17:34:28,50831,12310,#PrayForNashville https://t.co/cja0lMG6VC,prayfornashville https co cja0lmg6vc,-1
1448,1234893805389844481,2020-03-03 17:30:01,40324,9603,It was my great honor to address the National ...,it was my great honor to address the national ...,+1
1449,1234869067892305923,2020-03-03 15:51:43,77590,18805,The Federal Reserve is cutting but must furthe...,the federal reserve is cutting but must furthe...,-1


In [0]:
dataset.describe()
